In [1]:
import sympy as sp
from sympy.abc import a
import autograd.numpy as np
import numpy as np2
from pint import UnitRegistry
ureg = UnitRegistry()

from compute import Var

In [2]:
from astropy import units as u

In [11]:
np2.arccos(1*u.m / (2*u.m))

<Quantity 1.04719755 rad>

In [5]:
from compute import Evaluable, get_unit

In [6]:
x1 = Var('a', unit='m')
x2 = Var('h', unit='m')

In [15]:
ureg.meter/(ureg.meter-ureg.meter)

TypeError: unsupported operand type(s) for -: 'Unit' and 'Unit'

In [20]:
ureg.dimensionless

<Unit('dimensionless')>

In [33]:
mypp = ureg.wraps(ureg.meter, (ureg.meter,ureg.meter))(lambda x,y: x*y*np2.arccos(x/y))

In [37]:
class MockFloat(float):
    def __new__(self, value):
        return float.__new__(self, value)
    def __init__(self, value):
        float.__init__(value)
    def __add__(self, other):
        return self
    def __sub__(self, other):
        return self
    def __mul__(self, other):
        return self
    def __truediv__(self, other):
        return self
    def __floordiv__(self, other):
        return self
    def __mod__(self, other):
        return self
    def __pow__(self, other):
        return self
    def __rshift__(self, other):
        return self
    def __lshift__(self, other):
        return self
    def __and__(self, other):
        return self
    def __or__(self, other):
        return self
    def __xor__(self, other):
        return self

In [47]:
f = Foo(10)
f2 = Foo(2)

In [48]:
ureg.Quantity(f, 'm')

10.0 <Unit('meter')>

In [49]:
np2.sqrt(ureg.Quantity(f, 'm**2'))

3.1622776601683795 <Unit('meter')>

In [56]:
(ureg.Quantity(f, 'm')/(ureg.Quantity(f2, 'cm')-ureg.Quantity(f2, 'm')))

10.0 <Unit('meter / centimeter')>

In [8]:
get_unit(sp.acos(x1/x2))

C:\Users\johan\Miniconda2\envs\mdo\lib\site-packages\pint\numpy_func.py:303: RuntimeWarning: invalid value encountered in arccos
  result_magnitude = func(*stripped_args, **stripped_kwargs)


nan <Unit('radian')>

In [15]:
x1 = Var('a', unit=ureg('m/s**2'))
x2 = Var('m', unit=ureg('g'))
value = Var('F', unit=ureg('N'))

In [16]:
eq = equation_with_unit(value, x1*x2)

In [19]:
eq.jfx(np.array([1.,1.]))

array([0.001, 0.001])

In [27]:
def get_unit(expr):
    if isinstance(expr, Var):
        return expr.varunit
    else:
        free_symbols = list(expr.free_symbols)
        fx = sp.lambdify(free_symbols, expr, anp_math)
        return fx(*(free_symbol.varunit for free_symbol in free_symbols))

In [12]:
def get_unit_multiplier(unit):
    return unit.to_base_units().magnitude

In [15]:
def equation_with_unit(left, right):
        orig_unit = get_unit(left) # could be None
        unit = orig_unit if orig_unit else ureg('')
        rhs_unit = get_unit(right)
        convert = np.array([get_unit_multiplier(free_symbol.varunit) for 
                free_symbol in list(right.free_symbols)])
        if orig_unit:
            assert(unit.dimensionality == rhs_unit.dimensionality)
            conversion_unit = unit
        else: # left was not declared with a unit
            if not hasattr(rhs_unit, 'units'):
                conversion_unit = ureg('')
            else:
                conversion_unit = ureg.Quantity(1, 
                    rhs_unit.to_base_units().units)
            if isinstance(left, sp.Symbol):
                left.varunit = conversion_unit

        factor = get_unit_multiplier(conversion_unit)

        def correction(fx):
            def rhsfx_corrected(*args):
                return fx(*(convert*np.array(args).flatten()))/factor
            return rhsfx_corrected

        return Equation(left, right, decorator=correction)

In [28]:
eq = equation_with_unit(value, x1*x2)

In [29]:
eq.fx(np.array([100., 1000.]))

1.0

In [15]:
expr = x1+x2
eq = Equation(value, expr)

In [33]:
unit = 'mm' # or None
orig_unit = unit
unit = ureg(unit) if unit else ureg('')
rhs = value
rhs_unit = eq.fx(*(free_symbol.varunit 
                for free_symbol in expr.free_symbols))

In [35]:
if orig_unit:
    assert(unit.dimensionality == rhs_unit.dimensionality)
    conversion_unit = unit
else:
    if not hasattr(rhs_unit, 'units'):
        conversion_unit = ureg('')
    else:
        conversion_unit = ureg.Quantity(1, 
            rhs_unit.to_base_units().units)

In [36]:
conversion_unit

1 <Unit('millimeter')>

In [37]:
factor = get_unit_multi(conversion_unit)

In [38]:
factor

0.001

In [39]:
free_symbols = list(expr.free_symbols)
convert = np.array([get_unit_multi(free_symbol.varunit) for 
                    free_symbol in free_symbols])

In [40]:
convert

array([0.01, 1.  ])

In [ ]:
def rhsfx_corrected(*args):
    return rhsfx(*(convert*np.array(args).flatten()))/factor

In [48]:
import autograd.numpy as anp
math_functions = ['cos', 'sin', 'tan', 'arccos', 'arcsin', 'arctan', 'sqrt', 
'exp', 'log', 'log2', 'log10']

anp_math = {elt: getattr(anp, elt) for elt in math_functions}

In [51]:
get_unit(x1*x2)

1 <Unit('centimeter * meter')>